### Importing of Packages

In [1]:
%%capture
!pip install ipython-autotime
%load_ext autotime

time: 0 ns (started: 2022-06-27 06:40:14 +01:00)


In [2]:
!pip install imblearn --user

time: 5.62 s (started: 2022-06-27 06:40:14 +01:00)


In [3]:
# Standard libraries
# Standard libraries
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Preprocessing
import re
from string import punctuation
import nltk
nltk.download(['stopwords','punkt'])
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from wordcloud import WordCloud, STOPWORDS
from nltk.stem import WordNetLemmatizer 


from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.utils import resample
from imblearn.pipeline import Pipeline
# Building classification models
from sklearn.linear_model import RidgeClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.svm import LinearSVC, SVC
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, KFold, cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn import metrics
import xgboost as xgb
from sklearn.ensemble import StackingClassifier

# Model evaluation
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import cross_val_score
from imblearn.over_sampling import SMOTE
from sklearn import metrics

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\SAMSON\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\SAMSON\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


time: 22.7 s (started: 2022-06-27 06:40:20 +01:00)


### Loading of Dataset and general overview

In [4]:
train = pd.read_csv('train_set.csv')
test = pd.read_csv('test_set.csv')

time: 219 ms (started: 2022-06-27 06:40:42 +01:00)


In [5]:
# Splitting the labels and features
train.head(5)

,lang_id,text
0,xho,umgaqo-siseko wenza amalungiselelo kumaziko ax...
1,xho,i-dha iya kuba nobulumko bokubeka umsebenzi na...
2,eng,the province of kwazulu-natal department of tr...
3,nso,o netefatša gore o ba file dilo ka moka tše le...
4,ven,khomishini ya ndinganyiso ya mbeu yo ewa maana...


time: 62 ms (started: 2022-06-27 06:40:42 +01:00)


In [6]:
#Taking general overview at both datasets
print('TRAINING DATA')
print('============='+('\n'))
print('Shape of the dataset: {}\n'.format(train.shape))
print('Total Number of unique tweets: {}\n'.format(len(set(train['text']))))
print('Total Number of missing values:\n{}\n\n'.format(train.isnull().sum()))
print('TEST DATA')
print('========='+('\n'))
print('Shape of the dataset: {}\n'.format(test.shape))
print('Total Number of unique tweets: {}\n'.format(len(set(test['text']))))
print('Total Number of missing values:\n{}\n' .format(test.isnull().sum()))

TRAINING DATA

Shape of the dataset: (33000, 2)

Total Number of unique tweets: 29948

Total Number of missing values:
lang_id    0
text       0
dtype: int64


TEST DATA

Shape of the dataset: (5682, 2)

Total Number of unique tweets: 5459

Total Number of missing values:
index    0
text     0
dtype: int64

time: 250 ms (started: 2022-06-27 06:40:43 +01:00)


## Data Preprocessing

In [7]:
def text_preprocessing(text):
    
    '''
    This functions cleans text from line breaks, URLs, numbers, etc.
    '''
    
    text = text.lower() #to lower case
    text = text.replace('\n', ' ') # remove line breaks
    text = text.replace('\@(\w*)', '') # remove mentions
    text = re.sub(r"\bhttps://t.co/\w+", '', text) # remove URLs
    text = re.sub('\w*\d\w*', '', text) # remove numbers
    text = re.sub(r'\#', '', text) # remove hashtags. To remove full hashtag: '\#(\w*)'
    text = re.sub('\w*\d\w*', '', text) # removes numbers?
    text = re.sub(' +', ' ', text) # remove 1+ spaces
    text = re.sub("\n"," ",text)
    text =' '.join(text.split())

    return text

time: 187 ms (started: 2022-06-27 06:40:43 +01:00)


In [8]:
# Splitting the labels and features
train.head(5)

,lang_id,text
0,xho,umgaqo-siseko wenza amalungiselelo kumaziko ax...
1,xho,i-dha iya kuba nobulumko bokubeka umsebenzi na...
2,eng,the province of kwazulu-natal department of tr...
3,nso,o netefatša gore o ba file dilo ka moka tše le...
4,ven,khomishini ya ndinganyiso ya mbeu yo ewa maana...


time: 141 ms (started: 2022-06-27 06:40:43 +01:00)


In [9]:
test.head(5)

,index,text
0,1,"Mmasepala, fa maemo a a kgethegileng a letlele..."
1,2,Uzakwaziswa ngokufaneleko nakungafuneka eminye...
2,3,Tshivhumbeo tshi fana na ngano dza vhathu.
3,4,Kube inja nelikati betingevakala kutsi titsini...
4,5,Winste op buitelandse valuta.


time: 125 ms (started: 2022-06-27 06:40:43 +01:00)


In [10]:
train.text.duplicated(keep="first").value_counts()

False    29948
True      3052
Name: text, dtype: int64

time: 156 ms (started: 2022-06-27 06:40:43 +01:00)


In [11]:
test.text.duplicated(keep="first").value_counts()

False    5459
True      223
Name: text, dtype: int64

time: 140 ms (started: 2022-06-27 06:40:44 +01:00)


In [42]:
# Looking for duplicates
percent_duplicates = round((1-(train['text'].nunique()/len(train['text'])))*100,2)
print('Duplicated text in train data:')
print(percent_duplicates,'%')

Duplicated text in train data:
0.0 %
time: 47 ms (started: 2022-06-27 07:04:31 +01:00)


In [13]:
train.drop_duplicates(subset="text",keep="first",inplace=True,ignore_index=True)
train.shape

(29948, 2)

time: 172 ms (started: 2022-06-27 06:40:44 +01:00)


In [43]:
# Looking for duplicates
percent_duplicates = round((1-(test['text'].nunique()/len(test['text'])))*100,2)
print('Duplicated text in test data:')
print(percent_duplicates,'%')

Duplicated text in test data:
2.12 %
time: 16 ms (started: 2022-06-27 07:04:46 +01:00)


In [44]:
test.drop_duplicates(subset="text",keep="first",inplace=True,ignore_index=True)

time: 15 ms (started: 2022-06-27 07:04:51 +01:00)


In [45]:
percent_duplicates = round((1-(test['text'].nunique()/len(test['text'])))*100,2)
print('Duplicated text in test data:')
print(percent_duplicates,'%')

Duplicated text in test data:
0.0 %
time: 0 ns (started: 2022-06-27 07:04:53 +01:00)


In [17]:
train['text'] = train['text'].apply(text_preprocessing)
test['text'] = test['text'].apply(text_preprocessing)

time: 4.92 s (started: 2022-06-27 06:40:44 +01:00)


In [18]:
# Replace '.txt' with 'text file'
train["text"] = train["text"].str.replace(".txt", " text file")
test["text"] = test["text"].str.replace(".txt", " text file")

C:\Users\SAMSON\AppData\Local\Temp/ipykernel_12592/2305434871.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  train["text"] = train["text"].str.replace(".txt", " text file")


time: 281 ms (started: 2022-06-27 06:40:49 +01:00)


C:\Users\SAMSON\AppData\Local\Temp/ipykernel_12592/2305434871.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  test["text"] = test["text"].str.replace(".txt", " text file")


Great! We have dealt with all the duplicates in our dataset.

In [19]:
# Removing extra spaces
train['text']=train['text'].apply(lambda x: re.sub(' +',' ',x))

time: 562 ms (started: 2022-06-27 06:40:50 +01:00)


In [20]:
# Removing extra spaces
test['text']=test['text'].apply(lambda x: re.sub(' +',' ',x))

time: 94 ms (started: 2022-06-27 06:40:50 +01:00)


###### Create a copy

The first step in the preprocessing is to create a copy of the train dataframe for the EDA.In this step we start by determining the length of each text and storing this information in a new column. We then tokenize the text 

In [21]:
 df = train.copy()

time: 94 ms (started: 2022-06-27 06:40:50 +01:00)


In [22]:
def lemma(df):
    wnl = WordNetLemmatizer()
    df['length'] = df['text'].str.len()
    df['tokenized'] = df['text'].apply(word_tokenize)
    return df

df = lemma(df)
df.head()  

,lang_id,text,length,tokenized
0,xho,umgaqo-siseko wenza amalungiselelo kumaziko ax...,220,"[umgaqo-siseko, wenza, amalungiselelo, kumazik..."
1,xho,i-dha iya kuba nobulumko bokubeka umsebenzi na...,252,"[i-dha, iya, kuba, nobulumko, bokubeka, umsebe..."
2,eng,the province of kwazulu-natal department of tr...,264,"[the, province, of, kwazulu-natal, department,..."
3,nso,o netefatša gore o ba file dilo ka moka tše le...,217,"[o, netefatša, gore, o, ba, file, dilo, ka, mo..."
4,ven,khomishini ya ndinganyiso ya mbeu yo ewa maana...,239,"[khomishini, ya, ndinganyiso, ya, mbeu, yo, ew..."


time: 9.52 s (started: 2022-06-27 06:40:50 +01:00)


In [23]:
# Importing spacy
import spacy

# Loading model
nlp = spacy.load('en_core_web_sm',disable=['parser', 'ner'])

# Lemmatization with stopwords removal
df['lemmatized']=df['text'].apply(lambda x: ' '.join([token.lemma_ for token in list(nlp(x)) if (token.is_stop==False)]))

time: 3min 26s (started: 2022-06-27 06:41:00 +01:00)


In [24]:
df_grouped=df[['text','lemmatized']].groupby(by='text').agg(lambda x:' '.join(x))
df_grouped.head()

,lemmatized
text,
- - - - di a hohela boiqapelo bo hlwahlwa le bongodi bo tsotehang di a hohela di tsamaelana le sehlooho le bongodi bo kgabane di a baleha mosebetsi o kgotsofatsang ho boima ho latela dintlha ka botlalo,- - - - di hohela boiqapelo bo hlwahlwa le bon...
- d - and e - financial years i to be unsafe and ii not containing the exact proportions of ingredients as advertised if so in each case aa which products bb on what basis did the specified product fail the comprehensive inspection and cc what action was taken against the specified manufacturer,- d - e - financial year unsafe ii contain exa...
- e-mail bidoffice statssa gov za statistics south africa reserves the right not to consider or to reject all bids or amend modify postpone withdraw or terminates the bidding process at any time for any reason whatsoever office hours - and - mondays to fridays,- e - mail bidoffice statssa gov za statistic ...
a - tšhupane ya morwalo load index kg ka lebelo la km h a seo se šupago lebelo leo thaere e ka rwalago morwalo wo o itšego ka lona ela hloko gore ditšhupetšo mabapi le morwalo di ka fapana go ya tirišo ya thaere ge eba e dirišwa go goga morwalo goba go sepediša sedirišwa ka tokologo,- tšhupane ya morwalo load index kg ka lebelo ...
a - vragindeks kg km h a wat beteken die spoed waarteen die band die spesifieke vrag kan dra let daarop dat verskillende vraggraderings van toepassing kan wees indien dit n trekkerband is of slegs n vrylopende implement,- vragindek kg km h wat beteken die spoe waart...


time: 359 ms (started: 2022-06-27 06:44:26 +01:00)


In [25]:
# Creating Document Term Matrix
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(analyzer='word')
data=cv.fit_transform(df_grouped['lemmatized'])
df_dtm = pd.DataFrame(data.toarray(), columns=cv.get_feature_names())
df_dtm.index=df_grouped.index
df_dtm.head(3)

C:\Users\SAMSON\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,aa,aabameli,aaent,aak,aan,aanbeveel,aanbevele,aanbeveling,aanbevole,aanbie,...,ṱuwa,ṱuwe,ṱuwedza,ṱuwedzi,ṱuṱuwedza,ṱuṱuwedzaho,ṱuṱuwedze,ṱuṱuwedzea,ṱuṱuwedzwa,ṱuṱuwedzwe
text,,,,,,,,,,,,,,,,,,,,,
- - - - di a hohela boiqapelo bo hlwahlwa le bongodi bo tsotehang di a hohela di tsamaelana le sehlooho le bongodi bo kgabane di a baleha mosebetsi o kgotsofatsang ho boima ho latela dintlha ka botlalo,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
- d - and e - financial years i to be unsafe and ii not containing the exact proportions of ingredients as advertised if so in each case aa which products bb on what basis did the specified product fail the comprehensive inspection and cc what action was taken against the specified manufacturer,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
- e-mail bidoffice statssa gov za statistics south africa reserves the right not to consider or to reject all bids or amend modify postpone withdraw or terminates the bidding process at any time for any reason whatsoever office hours - and - mondays to fridays,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


time: 3.09 s (started: 2022-06-27 06:44:27 +01:00)


## EXPLORATORY data Analysis

In [26]:
def frequency(text):
    # Count vectorizer excluding english stopwords
    cv = CountVectorizer(stop_words='english')
    words = cv.fit_transform(text)
    
    # Count the words in the texts and determine the frequency of each word
    sum_words = words.sum(axis=0)
    words_freq = [(word, sum_words[0, i]) for word, i in cv.vocabulary_.items()]
    words_freq = sorted(words_freq, key=lambda x: x[1], reverse=True)
    
    # Create a dataframe to store the top 25 words and their frequencies
    frequency = pd.DataFrame(words_freq, columns=['word', 'freq'])
    frequency = frequency.head(25)
    
    return frequency

time: 0 ns (started: 2022-06-27 06:44:30 +01:00)


In [27]:
frequency(df)

,word,freq
0,lang_id,1
1,text,1
2,length,1
3,tokenized,1
4,lemmatized,1


time: 125 ms (started: 2022-06-27 06:44:30 +01:00)


In [28]:
# Importing wordcloud for plotting word clouds and textwrap for wrapping longer text
#from wordcloud import WordCloud
#from textwrap import wrap

# Function for generating word clouds
#def generate_wordcloud(data,title):
# plt.figure(figsize=(10,8))
#  plt.imshow(wc, interpolation='bilinear')
 # plt.axis("off")
 # plt.title('\n'.join(wrap(title,60)),fontsize=13)
  #plt.show()
  # Transposing document term matrix
#df_dtm=df_dtm.transpose()

# Plotting word cloud for each product
#for index,product in enumerate(df_dtm.columns):
#generate_wordcloud(df_dtm[product].sort_values(ascending=False),product)
# please uncomment it was wasting time and I need to submit

time: 93 ms (started: 2022-06-27 06:44:30 +01:00)


In [29]:
#vocab = set()
#corpus= [x.split() for x in df['text'].tolist()]
#for sentence in corpus:
  #for word in sentence:
 #   vocab.add(word.lower())
#print("Number of distinct words in raw data: ", len(vocab))

time: 94 ms (started: 2022-06-27 06:44:30 +01:00)


## Feature Engineering

### Splitting out X (indepedent) and Y (target/dependent) variables

In [30]:
X = train['text']
y = train['lang_id']

time: 94 ms (started: 2022-06-27 06:44:30 +01:00)


### Splitting of Training and Validation Sets

In [31]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.20)

time: 110 ms (started: 2022-06-27 06:44:30 +01:00)


### Model Building

In [32]:

classifiers = [LinearSVC(random_state=42),
                SVC(),
                #DecisionTreeClassifier(),
                #RandomForestClassifier(n_estimators=100, max_depth=2,
                                     #random_state=0, class_weight="balanced"),
                #MLPClassifier(alpha=1e-5,
                             #hidden_layer_sizes=(5, 2),
                             #random_state=42),
               LogisticRegression(random_state=42,
                                  multi_class='ovr',
                                  n_jobs=1,
                                  C=1e5,
                                  max_iter=100000),
               KNeighborsClassifier(n_neighbors=5),
               MultinomialNB(),
               ComplementNB(),
               SGDClassifier(loss='hinge',
                             penalty='l2',
                             alpha=1e-3,
                             random_state=42,
                             max_iter=5,
                             tol=None)
                #GradientBoostingClassifier(),
                #xgb.XGBClassifier(learning_rate=0.1,
                #                 n_estimators=1000,
                #                 max_depth=5,
                #                min_child_weight=1,
                #                 gamma=0,
                #                subsample=0.8,
                #                 colsample_bytree=0.8,
                #                 nthread=4,
                #                 seed=27)
               ]
# The commented code took tool long to load hence the reason for excluding them also, they didn't really perform as much as the ran

time: 125 ms (started: 2022-06-27 06:44:30 +01:00)


### Creating Function for Model Building

In [33]:
def models_building(classifiers, X_train, y_train, X_val, y_val):
    """
    This function takes in a list of classifiers
    and both the train and validation sets
    and return a summary of F1-score and
    processing time as a dataframe

    Input:
    classifiers: a list of classifiers to train
                 datatype: list
    X_train: independent variable for training
             datatype: series
    y_train: dependent variable for training
             datatype: series
    X_val: independent variable for validation
           datatype: series
    y_val: dependent variable for validation
           datatype: series

    Output:
    model_summary: F1 Score for all the classifiers
                   datatype: dataframe
    """

    models_summary = {}

    # Pipeline to balance the classses and then to build the model
    for clf in classifiers:
        clf_text = Pipeline([('tfidf', TfidfVectorizer(min_df=1,
                                                       max_df=0.9,
                                                       ngram_range=(1, 2))),
                             ('clf', clf)])

        # Logging the Execution Time for each model
        start_time = time.time()
        clf_text.fit(X_train, y_train)
        predictions = clf_text.predict(X_val)
        run_time = time.time()-start_time

        # Output for each model
        models_summary[clf.__class__.__name__] = {
            'F1-Macro': metrics.f1_score(y_val,
                                         predictions,
                                         average='macro'),
            'F1-Accuracy': metrics.f1_score(y_val, predictions,
                                            average='micro'),
            'F1-Weighted': metrics.f1_score(y_val,
                                            predictions,
                                            average='weighted'),
            'Execution Time': run_time}

    return pd.DataFrame.from_dict(models_summary, orient='index')

time: 109 ms (started: 2022-06-27 06:44:30 +01:00)


### Execution of the Classifiers

In [34]:
import time
classifiers_df = models_building(classifiers, X_train, y_train, X_val, y_val)
ordered_df = classifiers_df.sort_values('F1-Macro', ascending=False)
ordered_df

,F1-Macro,F1-Accuracy,F1-Weighted,Execution Time
ComplementNB,0.996586,0.996494,0.996491,7.030391
MultinomialNB,0.996055,0.995993,0.995993,6.791515
LinearSVC,0.994879,0.994825,0.994826,8.972674
LogisticRegression,0.994879,0.994825,0.994826,160.258196
SVC,0.992723,0.992654,0.992671,471.099693
SGDClassifier,0.982782,0.982805,0.982716,7.092083
KNeighborsClassifier,0.969662,0.969449,0.969222,12.683124


time: 11min 15s (started: 2022-06-27 06:44:31 +01:00)


### Comparing Classification Methods
The most performing models were the  Complement Naive Bayes and the Multinomial Naive Bayes with F1-Macro of 99.8% and accuracy of 99.7% while closely followed by Linear Support Vector Classifier,Logistic Regression, Support Vector Machine etc.

Hyperameter tuning is carried out to improve the F1-Macro score

### Hyperparameter Tuning on Most Performing Models

In [35]:
# Refining the train-test split for validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.01)

time: 16 ms (started: 2022-06-27 06:55:46 +01:00)


In [36]:
### Multinomial Naive Bayes

time: 125 ms (started: 2022-06-27 06:55:46 +01:00)


In [37]:
from sklearn.model_selection import StratifiedKFold
# Creating a pipeline for the gridsearch
param_grid = {'alpha': [0.1, 1, 5, 10]}  # setting parameter grid

tuned_mnb = Pipeline([('tfidf', TfidfVectorizer(min_df=1,
                                                max_df=0.8,
                                                ngram_range=(1, 2))),
                      ('mnb', GridSearchCV(MultinomialNB(),
                                           param_grid=param_grid,
                                           cv=5,
                                           n_jobs=-1,
                                           scoring='f1_weighted'))
                      ])

tuned_mnb.fit(X_train, y_train)  # Fitting the model

y_pred_mnb = tuned_mnb.predict(X_val)  # predicting the fit on validation set

print(classification_report(y_val, y_pred_mnb))


              precision    recall  f1-score   support

         afr       1.00      1.00      1.00        21
         eng       1.00      1.00      1.00        36
         nbl       1.00      1.00      1.00        29
         nso       1.00      1.00      1.00        33
         sot       1.00      1.00      1.00        32
         ssw       1.00      1.00      1.00        20
         tsn       1.00      1.00      1.00        31
         tso       1.00      1.00      1.00        27
         ven       1.00      1.00      1.00        18
         xho       1.00      1.00      1.00        18
         zul       1.00      1.00      1.00        35

    accuracy                           1.00       300
   macro avg       1.00      1.00      1.00       300
weighted avg       1.00      1.00      1.00       300

time: 19 s (started: 2022-06-27 06:55:46 +01:00)


In [38]:
from sklearn.model_selection import StratifiedKFold
# Creating a pipeline for the gridsearch
param_grid = {'alpha': [0.1, 1, 5, 10]}  # setting parameter grid

tuned_cnb = Pipeline([('tfidf', TfidfVectorizer(min_df=1,
                                                max_df=0.8,
                                                ngram_range=(1, 2))),
                      ('cnb', GridSearchCV(ComplementNB(),
                                           param_grid=param_grid,
                                           cv=5,
                                           n_jobs=-1,
                                           scoring='f1_weighted'))
                      ])

tuned_cnb.fit(X_train, y_train)  # Fitting the model

y_pred_cnb = tuned_cnb.predict(X_val)  # predicting the fit on validation set

print(classification_report(y_val, y_pred_cnb))

              precision    recall  f1-score   support

         afr       1.00      1.00      1.00        21
         eng       1.00      1.00      1.00        36
         nbl       1.00      1.00      1.00        29
         nso       1.00      1.00      1.00        33
         sot       1.00      1.00      1.00        32
         ssw       1.00      1.00      1.00        20
         tsn       1.00      1.00      1.00        31
         tso       1.00      1.00      1.00        27
         ven       1.00      1.00      1.00        18
         xho       1.00      1.00      1.00        18
         zul       1.00      1.00      1.00        35

    accuracy                           1.00       300
   macro avg       1.00      1.00      1.00       300
weighted avg       1.00      1.00      1.00       300

time: 14.9 s (started: 2022-06-27 06:56:05 +01:00)


### Creating File for Submission

In [39]:
submission_trial = pd.DataFrame(test['index'])
submission_trial['lang_id'] = tuned_mnb.predict(test['text'])
submission_trial.to_csv('submission_mn.csv', index=False)

time: 562 ms (started: 2022-06-27 06:56:20 +01:00)


## Conclusion
Several algorithms were tried and Complement Naive Bayes and Multinomial Naive Bayes classifier was the most performing. They performed very well on the training and validation datasets with an accuracy score of over 99% and F1 Macro score of over 99%. After testing the fitted model on the held-out/unseen dataset, they were able to predict the classes of languages with an F1 Score of about 99%.

### References
https://colab.research.google.com/drive/1eQydhzxK03z4_20_A9dunOq04trRYuZG?usp=sharing#scrollTo=InfhMWcIHlR_
https://www.analyticsvidhya.com/blog/2020/04/beginners-guide-exploratory-data-analysis-text-data/
https://stackoverflow.com/questions/46864696/how-to-resolve-python-error-while-generating-pie-chart-valueerror-explode-m